In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Installing and Importing important libraries

In [12]:
!pip install praw asyncpraw
!pip install emoji nest_asyncio


In [13]:
import asyncpraw, pandas as pd, re, emoji, nltk, asyncio
from nltk.corpus import stopwords

In [14]:
KEYWORDS  = ["depressed", "addiction help", "overwhelmed", "suicidal",
            "anxious", "panic attack", "lonely", "mental health",
            "self-harm", "substance abuse", "hopeless", "despair" , "trauma", "Drugs" , "betrayal"]

In [15]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()
import nest_asyncio
import asyncpraw, pandas as pd, re, emoji, nltk, asyncio
from nltk.corpus import stopwords

nltk.download('stopwords')
nest_asyncio.apply()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Reddit API info.....
REDDIT_CLIENT_ID = "0dsKwNeeAtMUZmppu2nKTw"
REDDIT_CLIENT_SECRET = "wZRN9A-DfdLWZWNOXlqbr6zljH62Jw"
REDDIT_USER_AGENT = "MyScript/1.0 (by u/Glittering-Spread293)"

In [17]:
async def init_reddit():
    return asyncpraw.Reddit(client_id=REDDIT_CLIENT_ID,
                            client_secret=REDDIT_CLIENT_SECRET,
                            user_agent=REDDIT_USER_AGENT)

async def extract_reddit(api, keywords):
    data = []
    subreddit = await api.subreddit("all")  # Await the coroutine

    for keyword in keywords:
        async for post in subreddit.search(keyword, limit=50):  # Awaited coroutine
            data.append({
                "ID": post.id,
                "Time": post.created_utc,
                "Text": post.selftext or post.title,
                "Likes": post.score,
                "Comments": post.num_comments,
                "Shares": 0
            })
    return pd.DataFrame(data)

In [18]:
def clean_text(text):
    text = emoji.replace_emoji(text, replace='')  # Remove emojis
    text = re.sub(r"http\S+|[^A-Za-z0-9\s]", '', text)  # Clean URLs & special chars
    return " ".join(w for w in text.lower().split() if w not in stopwords.words('english'))
async def main():
    reddit = await init_reddit()
    data = await extract_reddit(reddit, KEYWORDS)
    data['Text'] = data['Text'].apply(clean_text)
    data.to_csv("reddit_data_build_on_kaggle.csv", index=False)
    print("Reddit data saved to CSV ")

await main()

/usr/local/lib/python3.10/dist-packages/asyncpraw/models/reddit/subreddit.py:3232: RuntimeWarning: coroutine 'SubredditHelper.__call__' was never awaited
  super().__init__(reddit, _data=_data)


Reddit data saved to CSV ✅
